In [1]:
import pygame, sys
from pygame.locals import *
import random, time

pygame.init()

FPS = 60
FramePerSec = pygame.time.Clock()

BLUE  = (0, 0, 255)
RED   = (255, 0, 0)
GREEN = (0, 255, 0)
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)

SCREEN_WIDTH = 400
SCREEN_HEIGHT = 600
SPEED = 5
SCORE = 0  
COINS_COLLECTED = 0  
COINS_NEEDED_FOR_SPEED = 5  

font_small = pygame.font.SysFont("Verdana", 20)

background = pygame.image.load("AnimatedStreet.png")
coin_image = pygame.image.load("coin.png")  

DISPLAYSURF = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Game")

class Enemy(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__() 
        self.image = pygame.image.load("Enemy.png")
        self.rect = self.image.get_rect()
        self.rect.center = (random.randint(40, SCREEN_WIDTH-40), 0)  

    def move(self):
        global SCORE
        self.rect.move_ip(0, SPEED)
        if self.rect.top > SCREEN_HEIGHT:  # Reset enemy position when it moves off-screen
            SCORE += 1
            self.rect.top = 0
            self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)

class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__() 
        self.image = pygame.image.load("Player.png")
        self.rect = self.image.get_rect()
        self.rect.center = (160, 520)

    def move(self):
        pressed_keys = pygame.key.get_pressed()
        if self.rect.left > 0 and pressed_keys[K_LEFT]:
            self.rect.move_ip(-5, 0)  # Move left if within screen bounds
        if self.rect.right < SCREEN_WIDTH and pressed_keys[K_RIGHT]:        
            self.rect.move_ip(5, 0)  # Move right if within screen bounds

class Coin(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.transform.scale(coin_image, (30, 30))  # Resize coin image
        self.rect = self.image.get_rect()
        self.respawn()

    def move(self):
        self.rect.move_ip(0, SPEED)
        if self.rect.top > SCREEN_HEIGHT:  # Reset coin position if it moves off-screen
            self.respawn()

    def respawn(self):
        self.rect.top = random.randint(-100, -30)  # Spawn coin above the screen
        self.rect.center = (random.randint(40, SCREEN_WIDTH-40), self.rect.top)
        self.value = random.choice([1, 3, 5])  # Assign a random value to the coin

    def check_collision(self):
        global COINS_COLLECTED, SPEED
        if self.rect.colliderect(P1.rect):  # Check if player collects the coin
            COINS_COLLECTED += self.value  
            self.respawn()
            
            if COINS_COLLECTED >= COINS_NEEDED_FOR_SPEED:
                SPEED += 0.5  # Increase speed after collecting enough coins
                COINS_COLLECTED = 0  # Reset coin count

P1 = Player()
E1 = Enemy()
C1 = Coin()

enemies = pygame.sprite.Group()
enemies.add(E1)

coins = pygame.sprite.Group()
coins.add(C1)

all_sprites = pygame.sprite.Group()
all_sprites.add(P1, E1, C1)

while True:
    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            sys.exit()

    DISPLAYSURF.blit(background, (0, 0))

    # Render and display game stats
    score_text = font_small.render(f"Score: {SCORE}", True, BLACK)
    coin_text = font_small.render(f"Coins: {COINS_COLLECTED}", True, BLACK)
    speed_text = font_small.render(f"Speed: {SPEED:.1f}", True, RED)  

    DISPLAYSURF.blit(score_text, (10, 10))
    DISPLAYSURF.blit(coin_text, (SCREEN_WIDTH - 100, 10))
    DISPLAYSURF.blit(speed_text, (SCREEN_WIDTH // 2 - 30, 10))  

    for entity in all_sprites:
        DISPLAYSURF.blit(entity.image, entity.rect)
        entity.move() if hasattr(entity, "move") else entity.update()  # Ensure move() is called only for entities that have it

    C1.check_collision()

    if pygame.sprite.spritecollideany(P1, enemies):  # Check for collision with enemies
        pygame.mixer.Sound('crash.wav').play()
        time.sleep(0.5)
                    
        DISPLAYSURF.fill(RED)
        game_over_text = font_small.render("Game Over", True, BLACK)
        DISPLAYSURF.blit(game_over_text, (150, 250))
        pygame.display.update()
        
        for entity in all_sprites:
            entity.kill()  # Remove all sprites from the game
        time.sleep(2)
        pygame.quit()
        sys.exit()        

    pygame.display.update()
    FramePerSec.tick(FPS)


pygame 2.6.1 (SDL 2.28.4, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

c:\Users\Nurdaulet\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
